# Lab 2: Data Quality - Identify and Handle missing values

## Data Quality:

Data Quality refers to the condition of data based on factors like accuracy, completeness, consistency, and reliability. High-quality data is essential for accurate analysis and decision-making.

## Issues with Data Quality

Common problems include missing values, duplicates, inconsistencies, outliers, and errors in data entry or collection, leading to flawed analyses and incorrect conclusions.

## Missing Values and Its Reasons

Missing data occurs when no value is stored for a variable in a dataset. Reasons include human error, data corruption, non-response in surveys, or system failures during data collection.

## Techniques to Deal with Missing Values
   - **Remove Rows**: Delete rows with missing values (if minimal).
   - **Imputation**: Replace missing values with mean, median, or mode.
   - **Prediction Models**: Use algorithms to predict and fill in missing values.
   - **Flagging**: Mark missing values as a separate category (for categorical data).

----

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
DATASET_PATH = "dataset/pima_diabetes.csv"

In [3]:
col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'Diabetic']
df= pd.read_csv(DATASET_PATH, header=0, names=col_names)
df.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,Diabetic
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


#### Identify duplicate values in a dataset

- using `duplicated()` function to identify duplicate rows in a dataset.
    - True for duplicate
    - false for unique rows. 
    
    If two or more rows refers to identical objects and the attribute vaue are exaclty similar then we can simply remove the duplicated rows.

- using `duplicated().sum()` function to count the number of duplicate rows in a dataset.

In [4]:
df.duplicated().sum()

np.int64(0)

Since, the sum of the duplicated rows is 0, we can say that there are no duplicate rows in the dataset.

## Identify Missing values in a dataset

#### First Identify the spots in the dataset where missing values are present:

- Mark missing values as "NaN" in  rows or columns of the dataset. 
  - Sum, count etc operations ignores NAN values. By using the `replace()` function of Pandas DataFrame we can mark the  missing values as "NAN" in each columns.

- Then we can use "isnull()" function to mark  all "NAN" values in the dataset as True and based on it we can count total number of missing values in each column. Then replace "0"  with "NAN"

In [5]:
df_new = df.copy(deep=True) 
# deep copy of the dataset means that a new copy of the data is made and any changes made to the new copy will not affect the original data
df_new[['glucose', 'bp', 'skin', 'insulin', 'bmi']] = df_new[['glucose', 'bp', 'skin', 'insulin', 'bmi']].replace(0, np.nan)

print("The table below shows we marked the feature value from glucose to bmi as NAN in missing fields in PIMA_NEW dataset")
print(df_new.isnull().sum())

The table below shows we marked the feature value from glucose to bmi as NAN in missing fields in PIMA_NEW dataset
pregnant      0
glucose       5
bp           35
skin        227
insulin     373
bmi          11
pedigree      0
age           0
Diabetic      0
dtype: int64


In [6]:
print(df_new.head())

   pregnant  glucose    bp  skin  insulin   bmi  pedigree  age  Diabetic
0         1     85.0  66.0  29.0      NaN  26.6     0.351   31         0
1         8    183.0  64.0   NaN      NaN  23.3     0.672   32         1
2         1     89.0  66.0  23.0     94.0  28.1     0.167   21         0
3         0    137.0  40.0  35.0    168.0  43.1     2.288   33         1
4         5    116.0  74.0   NaN      NaN  25.6     0.201   30         0


## Handle Missing values

There are many ways to deal with missing values. But the goal is whatever approach we take, our decision must be accurate or as close to accuracy as if there were real data values in the missing part.

#### 1. Eliminate rows containing missing values

Though this approach is not suitable in many practical cases, it is preferred if only a few rows (that represents each object in a data set) have missing values. However its impractical to remove the rows when most records are missing.   

> For eliminating rows with missing values use "object.dropna()" method but it requires all missing values to be replaced by "NAN" first which we have already done previously.

In [7]:
df[['glucose','bp','skin','insulin','bmi']] = df[['glucose','bp','skin','insulin','bmi']].replace(0, np.nan)
df.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,Diabetic
0,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
1,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
2,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
3,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
4,5,116.0,74.0,NaN,NaN,25.6,0.201,30,0


In [8]:
clean_data  = df.dropna()  # eliminate rows containing missing values
clean_data.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,Diabetic
2,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
3,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
5,3,78.0,50.0,32.0,88.0,31.0,0.248,26,1
7,2,197.0,70.0,45.0,543.0,30.5,0.158,53,1
12,1,189.0,60.0,23.0,846.0,30.1,0.398,59,1


In [9]:
clean_data.shape  ## now the daatset contains only 392 samples

(392, 9)

#### 2. Replace missing values with a test statistic

Missing values can be replaced by mean, median, quartiles or based on the type and nature of attribute values i.e whether the attribute or the column data is continuous, categorical or the similarity values of the observed data. 

It is also equally important to take into account the effect on accuracy of the learning algorithm based on the imputation approach.

<font color = green>From the histogram of exploratory data analysis, "glucose", "bmi", "skin" features are normally distributed  so we replace mean value in the missing elelemnt part while  "insulin" and "bp" are skewed so we replace meadian value in the missing part.  </font>
##### Filling the mean and median value  according to corresponding histogram distribution in the missing part  using fillna() method

#### Impute missing values through measures of central tendency based on feature histogram of lab 1 EDA

In [10]:
df_new.fillna({
    "glucose": df_new['glucose'].mean(),
    "bp": df_new['bp'].median(),

    "skin": df_new['skin'].mean(),
    "insulin": df_new['insulin'].median(),

    "bmi": df_new['bmi'].median()
}, inplace=True)

In [11]:
df_new   # show newly imputed values in corresponsing misisng place as a result of above code

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,Diabetic
0,1,85.0,66.0,29.000000,125.0,26.6,0.351,31,0
1,8,183.0,64.0,29.142593,125.0,23.3,0.672,32,1
2,1,89.0,66.0,23.000000,94.0,28.1,0.167,21,0
3,0,137.0,40.0,35.000000,168.0,43.1,2.288,33,1
4,5,116.0,74.0,29.142593,125.0,25.6,0.201,30,0
...,...,...,...,...,...,...,...,...,...
762,10,101.0,76.0,48.000000,180.0,32.9,0.171,63,0
763,2,122.0,70.0,27.000000,125.0,36.8,0.340,27,0
764,5,121.0,72.0,23.000000,112.0,26.2,0.245,30,0
765,1,126.0,60.0,29.142593,125.0,30.1,0.349,47,1


## Save cleaned dataset

In [12]:
OUTPUT_PATH = "dataset/imputed_data_diabetes.csv"

df_new.to_csv(OUTPUT_PATH, index=False)